In [6]:
Pkg.add("Gym")

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
Fetching: [========================================>]  99.9 %0.0 %12.3 %                        ]  38.7 %=====================>                   ]  51.0 %Fetching: [=====================>                   ]  51.3 %>              ]  63.5 %===============================>         ]  75.8 %]  88.0 %>    ]  89.6 % Resolving package versions...
 Installed MbedTLS ─ v0.6.4
  Updating `~/.julia/environments/v1.0/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.0/Manifest.toml`
  [739be429] ↑ MbedTLS v0.6.3 ⇒ v0.6.4
  Building MbedTLS → `~/.julia/packages/MbedTLS/CwGUN/deps/build.log`


In [7]:
Pkg.add("Flux")

 Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.0/Manifest.toml`
 [no changes]


In [14]:
using Pkg
using Flux
using Gym

┌ Info: Precompiling Flux [587475ba-b771-5e3f-ad9e-33799f191a9c]
└ @ Base loading.jl:1186


In [8]:
env = GymEnv("BipedalWalker-v2");

In [12]:
s_n = env.observation_space.shape[1]; # continuous state space
a_n = env.action_space.shape[1]; # continuous action space
ha_n = [400,400];
hc_n = [300, 300]

In [ ]:
function weightSync(target_net, source_net, tau=0.001):
    for i in size(m.layers):
        target_net.layers[i].W = (1-tau)*target_net.layers[i].W + tau*source_net.layers[i].W;
    end

In [3]:
struct Replay
    capacity::Float32
    memory::Float32
    position::Float32
end

In [6]:
Replay(1000,10000,0)

Replay(1000.0f0, 10000.0f0, 0.0f0)

In [ ]:
struct Replay:
    capacity::Float32
    memory
    position::Float32
end

function initialize(r::Replay, init_len, env):
    i = 0;
    r.action_dim = env.action_space.shape[1]
    while i < init_len:
        s = reset!(env)
        d = False
        while not d:
            a = rand(env.action_space.shape[1]).*(2).-1;
            ns, r, d, _ = step!(env, a);
            r.memory.append!((s,a,r,ns,d));
            s = ns;
            i +=1;
        end
    end
end

function push(r::Replay, transition):
    if length(r.memory)<=r.position:
        r.memory.append!(None)
    end
    r.memory[r.position]=transition;
    r.position = mod((r.position+1),r.capacity)
end

function sample(r::Replay, N):
    return sample(r.memory,N)
            

In [ ]:
struct OnrsteinUhlenbeckProcess
    theta::Float32
    mu::Float32
    sigma::Float32
    dt::Float32
    action_size
    x
end

function step(OUP::OnrsteinUhlenbeckProcess)
    xx = OUP.theta*([OUP.mu]*OUP.action_size-
        OUP.x[-OUP.action_size:])*OUP.dt + 
        OUP.sigma*randn(zeros(OUP.action_size))
    OUP.x = append!(OUP.x,xx)
    return OUP.x[-OUP.action_size:]

In [15]:
actor = Chain(
    Dense(s_n, ha_n[1], relu),
    BatchNorm(ha_n[1]),
    Dense(ha_n[1], ha_n[2], relu),
    BatchNorm(ha_n[2]),
    Dense(ha_n[2], a_n),
    tanh)

critic = Chain(
    Dense(s_n,hc_n[1],relu),
    BatchNorm(hc_n[1]),
    Dense(hc_n[1]+a_n, hc_n[2],relu),
    Dense(hc_n[2],1)
    )

Chain(Dense(24, 24, tanh), Dense(24, 24, tanh), Dense(24, 4))

In [29]:
length(env.action_space.shape)

1

In [26]:
rand(3,3).*(2).-1

3×3 Array{Float64,2}:
 -0.050383    0.360395   0.984606
  0.0547567  -0.224643   0.585206
  0.791885    0.373463  -0.857316

In [11]:
episode_count = 2000;
min_epsilon = 0.01;
decay_rate = 5/episode_count;

0.0912641486376598

In [2]:
# General Algorithm for Q learning

# For each step in each episode: 
#     Use policy defined by training network to pick actions
#     Update experience replay
# After each episode: 
# Sample from experience replay
# Update training net; (update target net after x episodes)

# using Gym
reward = 0;
episode_count = 2000;
min_epsilon = 0.01;
decay_rate = 5/episode_count;

for i=1:episode_count
    # set epsilon for exploring policy
    epsilon = min_epsilon + (1.0 - min_epsilon)*exp(-decay_rate*i)
    total = 0
    ob = reset!(env)
#     render(env)#comment out this line if you do not want to visualize the environment
    while true
        Q_values = m(ob)
        # exploration
        if epsilon > Random.rand()
            action = Random.rand(env.action_space.shape[1])
        else
            # take max of Q_values
#         action = sample(env.action_space)
        end
        ob, reward, done, information = step!(env, action)
        total += reward
#         render(env)#comment out this line if you do not want to visualize the environment
        done && break
    end
    # Experience reply
    # update training net 
    i%10 && 
    println("episode $i total Rewards: $total")
end

episode 1 total Rewards: -85.97788289192233
episode 2 total Rewards: -98.68037809058097
episode 3 total Rewards: -92.41382523298962
episode 4 total Rewards: -99.93802021494935
episode 5 total Rewards: -79.69650269939272
episode 6 total Rewards: -102.57114505678385
episode 7 total Rewards: -110.65297324995726
episode 8 total Rewards: -100.0879210012217
episode 9 total Rewards: -98.46423048953291
episode 10 total Rewards: -85.04734643681674


In [13]:
sample(env.action_space)

4-element Array{Float64,1}:
  0.5131067509384377 
  0.8626462959402228 
 -0.3850172680270889 
  0.40907506858505505

In [30]:
env.action_space.shape[1]

4

In [31]:
Random.rand(env.action_space.shape[1])

4-element Array{Float64,1}:
 0.5091860357094704 
 0.46913460209260993
 0.03589676414598042
 0.27638164170088775

In [ ]:
# State consists of hull angle speed, angular velocity, horizontal speed, vertical speed,
# position of joints and joints angular speed, legs contact with ground, and 10 lidar
# rangefinder measurements to help to deal with the hardcore version. There's no coordinates
# in the state vector. Lidar is less useful in normal version, but it works.

In [5]:
env.observation_space

BoxS(Float32[-Inf, -Inf, -Inf, -Inf, -Inf, -Inf, -Inf, -Inf, -Inf, -Inf  …  -Inf, -Inf, -Inf, -Inf, -Inf, -Inf, -Inf, -Inf, -Inf, -Inf], Float32[Inf, Inf, Inf, Inf, Inf, Inf, Inf, Inf, Inf, Inf  …  Inf, Inf, Inf, Inf, Inf, Inf, Inf, Inf, Inf, Inf], (24,))

In [3]:
env.action_space
# I think this is force on 4 joints
# limits: -1 to 1

BoxS(Float32[-1.0, -1.0, -1.0, -1.0], Float32[1.0, 1.0, 1.0, 1.0], (4,))

In [44]:
env.action_space.sample()

ErrorException: type BoxS has no field sample